In [1]:
import pandas as pd
import librosa
import librosa.display as dsp
from IPython.display import Audio

In [2]:
import numpy as np
import os
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder

In [3]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [4]:
import tensorflow as tf

In [4]:
df1 = pd.read_csv('data/5차년도.csv', encoding='cp949')
df2 = pd.read_csv('data/5차년도_2차.csv', encoding='cp949')

In [5]:
df = pd.concat([df1,df2], ignore_index=True)

,wav_id,발화문,상황,1번 감정,1번 감정세기,2번 감정,2번 감정세기,3번 감정,3번 감정세기,4번 감정,4번감정세기,5번 감정,5번 감정세기,나이,성별
0,5ed10dbc2880d70f286121c3,개를 예쁘다고 사놓고 끝까지 키우지도 않고 버리는 사람들이 엄청 많아졌대.,disgust,Angry,2,Angry,2,Angry,2,Angry,2,Angry,2,33,female
1,5ecb60ef9aa8ea0eec53edb1,지금도 그대로 있어. 치우는 사람이 없어.,disgust,Neutral,0,Disgust,2,Sadness,2,Disgust,2,Disgust,1,48,female
2,5f052858b140144dfcfef768,맞아. 무기력증인 것 같아. 한동안 정말 바빴었거든.,sad,Sadness,2,Sadness,1,Sadness,2,Sadness,2,Sadness,2,48,female
3,5f0e72c4b140144dfcff3fa5,오늘이 발표날인데 연락이 없더라고. 그래서 알아봤더니 명단에 내 이름이 없대.,sad,Sadness,1,Sadness,2,Sadness,2,Sadness,2,Sadness,2,48,female
4,5ed10ddd7e21a10eee2537ce,그치. 개 키우는 사람이 늘어나니까 그만큼 버리는 사람도 늘어나는 거야!,disgust,Angry,2,Angry,2,Angry,2,Angry,2,Sadness,2,33,female
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29380,5fbe313c44697678c497c05a,나 엘리베이터에 갇혔어.,fear,happiness,1,sadness,1,sadness,2,sadness,1,sadness,1,23,female
29381,5fbe251044697678c497bfb8,하지만 기분이 나쁜 걸 어떡해?,angry,sadness,1,fear,1,sadness,2,sadness,1,neutral,0,23,female
29382,5fbe31584c55eb78bd7cee7f,자취방 엘리베이턴데 정전인가봐.,fear,sadness,1,neutral,0,sadness,2,fear,1,sadness,1,23,female
29383,5fbe2f8544697678c497c047,나 드디어 프로젝트 끝났어!,happiness,disgust,1,sadness,1,neutral,0,happiness,1,sadness,1,23,female


In [6]:
wavid = []
for i in df['wav_id']:
    wavid.append(i+'.wav')
wavid

['5ed10dbc2880d70f286121c3.wav',
 '5ecb60ef9aa8ea0eec53edb1.wav',
 '5f052858b140144dfcfef768.wav',
 '5f0e72c4b140144dfcff3fa5.wav',
 '5ed10ddd7e21a10eee2537ce.wav',
 '5f0e72e1b140144dfcff3fa6.wav',
 '5ed10e039aa8ea0eec53f0ee.wav',
 '5f01ce9510dbee4d2f789ad8.wav',
 '5f0e72fbb140144dfcff3fa7.wav',
 '5ed10e31c90a530ee56b5898.wav',
 '5ecb613b79bf120ed2b80ee4.wav',
 '5f07bffeb140144dfcff09a8.wav',
 '5f0e7342b140144dfcff3fa9.wav',
 '5f0fa5e6b140144dfcff4734.wav',
 '5f07c011b140144dfcff09aa.wav',
 '5f0e7377b140144dfcff3fac.wav',
 '5eff3d35704f492ee125534c.wav',
 '5f07c028b140144dfcff09ad.wav',
 '5f0e738eb140144dfcff3fad.wav',
 '5f0e73a6b140144dfcff3fae.wav',
 '5ed979dc7e21a10eee253e90.wav',
 '5f01cf40b140144dfcfecec6.wav',
 '5f07c069b140144dfcff09b3.wav',
 '5f0c79c3b140144dfcff2f76.wav',
 '5f0e73c3b140144dfcff3faf.wav',
 '5f07c099b140144dfcff09bc.wav',
 '5f0c79dab140144dfcff2f77.wav',
 '5f0e73f8b140144dfcff3fb1.wav',
 '5ef7f6a6497740547350bcd9.wav',
 '5ef74a52497740547350bb84.wav',
 '5efa05fc

In [10]:
le = LabelEncoder()
le.fit(df['상황'])
labels = le.transform(df['상황'])
le.classes_

array(['anger', 'angry', 'disgust', 'fear', 'happiness', 'neutral', 'sad',
       'sadness', 'surprise'], dtype=object)

In [11]:
df['id'] = wavid
df['label'] = labels

In [12]:
df = df[['id','상황','label']]

In [13]:
df

,id,상황,label
0,5ed10dbc2880d70f286121c3.wav,disgust,2
1,5ecb60ef9aa8ea0eec53edb1.wav,disgust,2
2,5f052858b140144dfcfef768.wav,sad,6
3,5f0e72c4b140144dfcff3fa5.wav,sad,6
4,5ed10ddd7e21a10eee2537ce.wav,disgust,2
...,...,...,...
29380,5fbe313c44697678c497c05a.wav,fear,3
29381,5fbe251044697678c497bfb8.wav,angry,1
29382,5fbe31584c55eb78bd7cee7f.wav,fear,3
29383,5fbe2f8544697678c497c047.wav,happiness,4


In [22]:
def wavdataset():
    dataset=[]
    wpath = 'data/5차_wav'
    for file in tqdm(os.listdir(wpath), colour='green'):
        if 'wav' in file:
            abs_file_path = os.path.join(wpath,file)
            data, sr = librosa.load(abs_file_path, sr= 16000)
            class_label = int(df[df.id == file].label)
            dataset.append([file, data, class_label])
    print('생성완료')
    return pd.DataFrame(dataset,columns=['id','data','class_label'])

In [23]:
wavdata = wavdataset()

100%|██████████████████████████████████████████████████████████████████████████| 29385/29385 [2:02:41<00:00,  3.99it/s]

생성완료


In [24]:
wavdata

,id,data,class_label
0,5ebd3dd7c38c123b9ec6deba.wav,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",6
1,5ebd3df2c38c123b9ec6debb.wav,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",6
2,5ebd3e398849ac3aacea582d.wav,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",6
3,5ebd3eb67bef803b4851e081.wav,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",6
4,5ebd3f0cf2fe7a3b1d545225.wav,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",6
...,...,...,...
29380,5fbe35bfcb503578af9eedb2.wav,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1
29381,5fbe361844697678c497c07e.wav,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",2
29382,5fbe364744697678c497c07f.wav,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",2
29383,5fbe3656cb503578af9eedb4.wav,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",2


In [25]:
from sklearn.model_selection import train_test_split

In [96]:
X_train, X_test, y_train, y_test = train_test_split(wavdata['data'],wavdata['class_label'], test_size=0.25, stratify=)

array([array([0.        , 0.        , 0.        , ..., 0.00022634, 0.00029589,
              0.00023341], dtype=float32)                                     ,
       array([ 0.        ,  0.        ,  0.        , ..., -0.00018122,
              -0.00020711,  0.        ], dtype=float32)               ,
       array([0.        , 0.        , 0.        , ..., 0.0003146 , 0.00040687,
              0.00049165], dtype=float32)                                     ,
       ...,
       array([0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ..., 1.2858022e-05,
              1.3675815e-05, 0.0000000e+00], dtype=float32)                   ,
       array([ 0.000000e+00,  0.000000e+00,  0.000000e+00, ..., -8.963979e-06,
               8.724056e-07,  9.298258e-06], dtype=float32)                   ,
       array([ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00, ...,
              -1.8387547e-05,  1.1083576e-05,  0.0000000e+00], dtype=float32)],
      dtype=object)

In [101]:
a = wavdata[wavdata.class_label == 0].index
b = wavdata[wavdata.class_label == 7].index

In [105]:
wavdata.drop(a,axis=0,inplace=True)
wavdata.drop(b,axis=0,inplace=True)

In [109]:
wavdata.reset_index(drop=True, inplace=True)

In [110]:
wavdata

,id,data,class_label
0,5ebd3dd7c38c123b9ec6deba.wav,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",6
1,5ebd3df2c38c123b9ec6debb.wav,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",6
2,5ebd3e398849ac3aacea582d.wav,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",6
3,5ebd3eb67bef803b4851e081.wav,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",6
4,5ebd3f0cf2fe7a3b1d545225.wav,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",6
...,...,...,...
23373,5fbe35bfcb503578af9eedb2.wav,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1
23374,5fbe361844697678c497c07e.wav,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",2
23375,5fbe364744697678c497c07f.wav,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",2
23376,5fbe3656cb503578af9eedb4.wav,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",2


In [ ]:
['angry', 'disgust', 'fear', 'happiness', 'neutral', 'sad','surprise']

In [113]:
wavdf = wavdata.replace({'class_label':{1:0, 2:1, 3:2, 4:3, 5:4, 6:5, 8:6}})

In [114]:
wavdf['class_label'].value_counts()

5    4860
3    4548
0    3263
4    3257
1    3145
2    2550
6    1755
Name: class_label, dtype: int64

In [116]:
x_train_data, x_test_data, y_train_data, y_test_data = train_test_split(
    wavdf['data'],wavdf['class_label'],test_size=0.25,stratify=wavdf['class_label'], random_state=12)

In [137]:
def preprocess_dataset(data):
    frame_length = 0.025
    frame_stride = 0.010
    sampling_rate = 16000
    input_nfft = int(round(sampling_rate*frame_length))
    input_stride = int(round(sampling_rate*frame_stride))
    train_mfcc=[]
    for i in tqdm(data):
        mfcc = librosa.feature.mfcc(y = i, sr = sampling_rate, n_mfcc=100,n_fft=input_nfft, hop_length=input_stride)
        mfcc = tf.image.resize(mfcc[:, :, np.newaxis],(100,100))
        train_mfcc.append(mfcc)
    return train_mfcc

In [138]:
X = np.array(preprocess_dataset(x_train_data))

100%|████████████████████████████████████████████████████████████████████████████| 17533/17533 [05:24<00:00, 54.07it/s]


In [143]:
X_test = np.array(preprocess_dataset(x_test_data))

100%|██████████████████████████████████████████████████████████████████████████████| 5845/5845 [01:36<00:00, 60.57it/s]


In [159]:
y = np.array(y_train_data)
y_test = np.array(y_test_data)

In [161]:
np.save('train.npy',X)
np.save('test.npy',X_test)
np.save('y_train.npy',y)
np.save('y_test.npy',y_test)

In [7]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from keras import layers

In [10]:
def CNN():
    inp = tf.keras.Input(shape=(X.shape[1], X.shape[2], 1))
    x = layers.Conv2D(32, 3, 1, padding="same")(inp)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.MaxPooling2D()(x)
    x = layers.Conv2D(64, 3, 1, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.MaxPooling2D()(x)
    x = layers.Conv2D(128, 3, 1, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.MaxPooling2D()(x)
    x = layers.Conv2D(256, 3, 1, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(64, activation="relu")(x)
    oup = layers.Dense(len(np.unique(y)), activation="softmax")(x)
    model = tf.keras.Model(inputs=inp, outputs=oup)
    return model

model = CNN()
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 100, 100, 1)]     0         
                                                                 
 conv2d_4 (Conv2D)           (None, 100, 100, 32)      320       
                                                                 
 batch_normalization_4 (Batc  (None, 100, 100, 32)     128       
 hNormalization)                                                 
                                                                 
 activation_4 (Activation)   (None, 100, 100, 32)      0         
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 50, 50, 32)       0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 50, 50, 64)        18496 

In [14]:
validations = []
predictions = []

idx=0

skf = StratifiedKFold(n_splits=10)
for train_index, val_index in tqdm(skf.split(X, y)):
    
    idx+=1

    X_train, X_val = X[train_index], X[val_index]
    y_train, y_val = y[train_index], y[val_index]
    
    train_ds = (
        tf.data.Dataset.from_tensor_slices((X_train, y_train))
        .shuffle(len(X_train))
        .batch(32)
        .prefetch(tf.data.experimental.AUTOTUNE)
    )

    val_ds = (
        tf.data.Dataset.from_tensor_slices((X_val, y_val))
        .batch(32)
        .prefetch(tf.data.experimental.AUTOTUNE)
    )

    model = CNN()

    lr = tf.keras.optimizers.schedules.CosineDecay(0.01, decay_steps=1000)
    optim = tf.keras.optimizers.SGD(learning_rate=lr, momentum=0.9)

    model.compile(
        optimizer=optim,
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"],
    )
    
    checkpoint_filepath=f"model_mfcc_{idx}.hdf5"

    checkpoint_callback = [
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=10,
            restore_best_weights=True,
        ),
        tf.keras.callbacks.ModelCheckpoint(
            checkpoint_filepath,
            monitor="val_loss",
            save_best_only=True,
            save_weights_only=True
        )
    ]

    history = model.fit(
        train_ds,
        validation_data=val_ds,
        epochs=50,
        callbacks=[checkpoint_callback],
    )

    validations.append(accuracy_score(y_val, np.argmax(model.predict(X_val), axis=1)))
    predictions.append(model.predict(X_test))

val_acc = np.mean(validations, axis=0)

print("validation_accuracy: ", val_acc)

0it [00:00, ?it/s]

Epoch 1/50
494/494 [==============================] - 146s 295ms/step - loss: 1.8316 - accuracy: 0.2847 - val_loss: 1.8306 - val_accuracy: 0.2851
Epoch 2/50
494/494 [==============================] - 166s 337ms/step - loss: 1.7339 - accuracy: 0.3219 - val_loss: 1.6982 - val_accuracy: 0.3375
Epoch 3/50
494/494 [==============================] - 165s 335ms/step - loss: 1.6956 - accuracy: 0.3375 - val_loss: 1.6876 - val_accuracy: 0.3472
Epoch 4/50
494/494 [==============================] - 165s 333ms/step - loss: 1.6980 - accuracy: 0.3372 - val_loss: 1.6876 - val_accuracy: 0.3472
Epoch 5/50
494/494 [==============================] - 170s 345ms/step - loss: 1.6965 - accuracy: 0.3358 - val_loss: 1.6878 - val_accuracy: 0.3472
Epoch 6/50
494/494 [==============================] - 174s 352ms/step - loss: 1.6973 - accuracy: 0.3381 - val_loss: 1.6878 - val_accuracy: 0.3472
Epoch 7/50
494/494 [==============================] - 172s 349ms/step - loss: 1.6973 - accuracy: 0.3365 - val_loss: 1.6875 -

1it [48:03, 2883.99s/it]

Epoch 1/50
494/494 [==============================] - 169s 341ms/step - loss: 1.8372 - accuracy: 0.2787 - val_loss: 1.8196 - val_accuracy: 0.2856
Epoch 2/50
494/494 [==============================] - 168s 339ms/step - loss: 1.7300 - accuracy: 0.3291 - val_loss: 1.7020 - val_accuracy: 0.3409
Epoch 3/50
494/494 [==============================] - 168s 341ms/step - loss: 1.6956 - accuracy: 0.3407 - val_loss: 1.6996 - val_accuracy: 0.3392
Epoch 4/50
494/494 [==============================] - 168s 339ms/step - loss: 1.6965 - accuracy: 0.3393 - val_loss: 1.6995 - val_accuracy: 0.3392
Epoch 5/50
494/494 [==============================] - 176s 357ms/step - loss: 1.6937 - accuracy: 0.3401 - val_loss: 1.6995 - val_accuracy: 0.3392
Epoch 6/50
494/494 [==============================] - 173s 350ms/step - loss: 1.6932 - accuracy: 0.3392 - val_loss: 1.6996 - val_accuracy: 0.3398
Epoch 7/50
494/494 [==============================] - 173s 350ms/step - loss: 1.6961 - accuracy: 0.3390 - val_loss: 1.6995 -

2it [1:38:06, 2953.72s/it]

...

Epoch 1/50
494/494 [==============================] - 161s 326ms/step - loss: 1.8347 - accuracy: 0.2815 - val_loss: 1.9602 - val_accuracy: 0.2042
Epoch 2/50
494/494 [==============================] - 163s 329ms/step - loss: 1.7309 - accuracy: 0.3241 - val_loss: 1.7091 - val_accuracy: 0.3343
Epoch 3/50
494/494 [==============================] - 163s 329ms/step - loss: 1.6967 - accuracy: 0.3368 - val_loss: 1.7045 - val_accuracy: 0.3377
Epoch 4/50
494/494 [==============================] - 163s 330ms/step - loss: 1.6968 - accuracy: 0.3372 - val_loss: 1.7045 - val_accuracy: 0.3383
Epoch 5/50
494/494 [==============================] - 165s 334ms/step - loss: 1.6966 - accuracy: 0.3376 - val_loss: 1.7045 - val_accuracy: 0.3366
Epoch 6/50
494/494 [==============================] - 165s 334ms/step - loss: 1.6967 - accuracy: 0.3373 - val_loss: 1.7047 - val_accuracy: 0.3371
Epoch 7/50
494/494 [==============================] - 166s 336ms/step - loss: 1.6969 - accuracy: 0.3369 - val_loss: 1.7046 -

8it [7:16:47, 3739.22s/it]

Epoch 1/50
494/494 [==============================] - 170s 343ms/step - loss: 1.8463 - accuracy: 0.2746 - val_loss: 1.8072 - val_accuracy: 0.3052
Epoch 2/50
494/494 [==============================] - 168s 341ms/step - loss: 1.7499 - accuracy: 0.3189 - val_loss: 1.7324 - val_accuracy: 0.3240
Epoch 3/50
494/494 [==============================] - 174s 352ms/step - loss: 1.7107 - accuracy: 0.3323 - val_loss: 1.7294 - val_accuracy: 0.3234
Epoch 4/50
494/494 [==============================] - 176s 357ms/step - loss: 1.7121 - accuracy: 0.3338 - val_loss: 1.7294 - val_accuracy: 0.3240
Epoch 5/50
 69/494 [===>..........................] - ETA: 2:30 - loss: 1.7322 - accuracy: 0.3062

8it [7:28:41, 3365.23s/it]


KeyboardInterrupt: 

In [5]:
X= np.load('train.npy')
y= np.load('y_train.npy')
X_test = np.load('test.npy')
y_test = np.load('y_test.npy')